In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
import os
import seaborn as sns
from matplotlib import pyplot as plt

In [105]:
strDataDir = '/project/hackathon/hackers09/hack095/NOVEL-CLINICAL-PREDICTION-APPROACHES-TO-MANAGING-CARE-FOR-ACUTE-PULMONARY-EMBOLISM-PATIENTS/'
strMergedCsv = os.path.join(strDataDir, 'enc_outcome_echo.csv')
strEchoLabeledCsv = os.path.join(strDataDir, 'echo_tag.csv')

In [106]:
dfMerged = pd.read_csv(strMergedCsv, index_col=0)
dfEchoLabeled = pd.read_csv(strEchoLabeledCsv, encoding='ISO-8859-1', index_col=0)

In [107]:
def preproc_labeled_echo(dfEchoLabeled):
    # Changing mlid to mild
    dfEchoLabeled['function'] = dfEchoLabeled['function'].replace('mlid', 'mild')
    dfEchoLabeled['dilation'] = dfEchoLabeled['dilation'].replace('mlid', 'mild')
    
    # na to 0, mild to 1, moderate to 2, severe to 3
    dictReplace = {'mild':1, 'moderate':2, 'severe':3}
    dfEchoLabeled['function'] = dfEchoLabeled['function'].fillna(0)
    dfEchoLabeled['function'] = dfEchoLabeled['function'].replace(dictReplace)

    dfEchoLabeled['dilation'] = dfEchoLabeled['dilation'].fillna(0)
    dfEchoLabeled['dilation'] = dfEchoLabeled['dilation'].replace(dictReplace)
    
    return dfEchoLabeled

In [108]:
dfEchoLabeled = preproc_labeled_echo(dfEchoLabeled)

In [109]:
def merge_labeled_echo(dfMerged, dfEchoLabeled):
    """
    Parse labeled echo data to keep matching enc, order_proc_id
    If multiple rows per enc and order id, keep max bc this 
    represents different line of report
    """
    srsIsNullNarrative = dfMerged['NARRATIVE_compiled'].isnull()
    for nEnc in dfMerged.index:
        nOrder = dfMerged.at[nEnc, 'ORDER_PROC_ID']
        dfTemp = dfEchoLabeled[dfEchoLabeled['HSP_ENC']==nEnc]
        dfTemp = dfTemp[dfTemp['ORDER_PROC_ID']==nOrder]
        if dfTemp.shape[0] > 0:    
            dfMerged.at[nEnc, 'echo_dilation'] = dfTemp['dilation'].max()
            dfMerged.at[nEnc, 'echo_function'] = dfTemp['function'].max()
        elif not srsIsNullNarrative[nEnc]:
            dfMerged.at[nEnc, 'echo_dilation'] = 0
            dfMerged.at[nEnc, 'echo_function'] = 0
    
    return dfMerged

In [110]:
dfMerged = merge_labeled_echo(dfMerged, dfEchoLabeled)

In [111]:
dfMerged.head()

,PATIENT_ID_x,ED_EPISODE_ID,ED_DISP,DISCH_DISP,ADT_ARRIVAL_TIME_DIFFSEC,ED_DISP_TIME_DIFFSEC,HOSP_DISCH_TIME_DIFFSEC,ADMIT_SOURCE,ADT_PAT_CLASS,HOSP_SERVICE,...,PATIENT_ID,ORDER_PROC_ID,NAME,ORDER_INST_DIFFSEC,PROC_START_TIME_DIFFSEC,RESULT_TIME_DIFFSEC,ECHO_TYPE,NARRATIVE_compiled,echo_dilation,echo_function
HSP_ENC,,,,,,,,,,,,,,,,,,,,,
260755660,1305,34382396,Admitted,To Home Or Self Care,-420,10500,627600,Home & Outside Location,Inpatient,Pulmonary,...,1305.0,330967554.0,CV ECHO,27060.0,27060.0,105180.0,echo_old,STUDY DATE: 04/08/2017 * REASON FOR STUDY: Sho...,0.0,0.0
192470437,1261,27636517,Admitted,To Home With Home Health,-120,9120,516300,Home & Outside Location,Inpatient,Cardiology,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
258754156,785,34189980,Admitted,Expired,0,40200,899640,Home & Outside Location,Inpatient,Cardiology,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306050512,504,38409274,Admitted,To Home Or Self Care,0,7860,388440,Home & Outside Location,Inpatient,Internal Medicine,...,504.0,355830397.0,CV ECHO,15060.0,15120.0,21000.0,echo_old,STUDY DATE: 06/25/2018 * REASON FOR STUDY: Tac...,0.0,0.0
163297609,631,22833060,Admitted,To Rehab Unit Zale Lipshy,0,18300,758880,Home & Outside Location,Inpatient,Hematology-Oncology,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
dfMerged.to_csv('./labeled_echo_merged.csv')